In [1]:
import torch
import pyro
import pyro.distributions as dist
import pyro.optim as optim
import logging
from torch.distributions import constraints
from bitcoin import BitcoinOTC
from pyro.infer import SVI, Trace_ELBO

# Validation checks
logging.basicConfig(format='%(message)s', level=logging.INFO)
pyro.enable_validation(True)

In [2]:
data = BitcoinOTC()

c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\andrew\anaconda3\envs\ppp\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
#sample_args = {'sample_str': 'random-walk', 'sample_args': {'start_num': 5, 'walk_length': 50}}
sample_args = {'sample_str': 'p-sampling', 'sample_args': {'sample_prob': 0.05}}

In [4]:
print(data.subsample(**sample_args))

{'node_ind': tensor([   0,    1,    2,  ..., 5982, 5992, 5996]), 'edge_ind': tensor([   54,    98,   111,  ..., 35508, 35557, 35585]), 'edge_list': tensor([[  30,   54,    3,  ..., 3450, 1017, 1952],
        [   0,    6,   65,  ..., 5982,  904, 5654]])}


In [5]:
omega_scale = 1.0
obs_scale = 1.0
embed_dim = 2

def guide(data, sample_args):
    r"""Defines a variational family to use to fit an approximate posterior
    distribution for the probability model defined in model.
    """
    # Parameters governing the priors on the embedding vectors
    omega_loc = pyro.param('omega_loc',
                           torch.randn(embed_dim, data.num_nodes))
    omega_scale = pyro.param('omega_scale', torch.tensor(1.0),
                             constraint=constraints.positive)

    # Parameters governing the prior for the linear regression
    beta_loc = pyro.param('beta_loc', 0.5*torch.randn(embed_dim))
    beta_scale = pyro.param('beta_scale', torch.tensor(1.0),
                            constraint=constraints.positive)
    mu_loc = pyro.param('mu_loc', torch.randn(1))
    mu_scale = pyro.param('mu_scale', torch.tensor(1.0),
                          constraint=constraints.positive)

    # Sample the coefficient vector and intercept for linear regression
    beta = pyro.sample('beta', dist.MultivariateNormal(
        loc=beta_loc, covariance_matrix=(beta_scale**2)*torch.eye(embed_dim)
    ))
    mu = pyro.sample('mu', dist.Normal(mu_loc, mu_scale))

    # Subsample vertices now in the guide
    subsample = data.subsample(**sample_args)

    # (Sub)sample embedding vectors
    for i in pyro.plate('nodes', size=data.num_nodes,
                        subsample=subsample['node_ind']):
        pyro.sample('omega_{}'.format(i),
                    dist.MultivariateNormal(
            loc=omega_loc[:, i],
            covariance_matrix=(omega_scale**2)*torch.eye(embed_dim)
        )
        )

    # Define plate for the edge subsampling to pass to model object
    with pyro.plate('edges', size=data.num_edges,
                    subsample=subsample['edge_ind']):
        # Note: we use pyro plate here as we need to keep the subsampling
        # persistent in both our model and guide functions, we so we
        # define an empty plate here just so we can use the same call
        # in the model.
        pass

    return beta, mu


def model(data, sample_args):
    r"""Defines a probabilistic model for the observed network data."""
    # Define priors on the regression coefficients
    mu = pyro.sample('mu', dist.Normal(torch.tensor(0.0), torch.tensor(2.0)))
    beta = pyro.sample('beta', dist.MultivariateNormal(
        loc=torch.zeros(embed_dim), covariance_matrix=4*torch.eye(embed_dim)
    ))

    # Define prior on the embedding vectors, do subsampling for the
    # embedding vector and then the likelihood term for the observed nodes
    omega = [0 for i in range(data.num_nodes)]

    for i in pyro.plate('nodes', size=data.num_nodes):
        # Embedding vectors
        omega[i] = pyro.sample('omega_{}'.format(i), dist.MultivariateNormal(
            loc=torch.zeros(embed_dim),
            covariance_matrix=(omega_scale**2)*torch.eye(embed_dim)
        ))

        # Draw Bernoulli, with or without data depending on if it is observed
        logit = mu + torch.dot(beta, omega[i])
        if i in data.nodes_train:
            pyro.sample('trust_{}'.format(i), dist.Bernoulli(logits=logit), obs=data.gt[i])

    # Draw terms corresponding to the edges
    for i in pyro.plate('edges', size=data.num_edges):
        logit_rating = 0.05 + 0.9*data.edge_weight[i]
        logit_rating = torch.log((logit_rating)/(1 - logit_rating))
        emip = torch.dot(omega[data.edge_index[0, i]],
                         omega[data.edge_index[1, i]])
        pyro.sample('a_{}'.format(i), dist.Normal(emip, obs_scale), obs=logit_rating)

In [6]:
import time

t0 = time.time()

svi = SVI(model,
          guide,
          optim.Adam({"lr": .05}),
          loss=Trace_ELBO())

t1 = time.time()
logging.info("Time to create SVI object: {}".format(t1-t0))

Time to create SVI object: 0.0


In [7]:
pyro.clear_param_store()
num_iters = 200

for i in range(num_iters):
    elbo = svi.step(data, sample_args)
    if i % 5 == 0:
        logging.info("Elbo loss: {}".format(elbo))
        if (i > 0):
            t2 = time.time()
            logging.info("Time to run 5 gradient steps of SVI: {}".format(t2-t1))
            t1 = time.time()

KeyboardInterrupt: 